In [4]:
!pip install ultralytics

ERROR: Operation cancelled by user
^C


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import sklearn as skl
from sklearn.model_selection import train_test_split
import cv2
import os
import random
%matplotlib inline
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score
import numpy as np

nombre_modelo = "yolov8n.pt"
epocas = 200
imgsz = 640
ruta_dataset = Path(r"/home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo")
yaml_path = ruta_dataset.parent / "data.yaml"


print("Ajustando etiquetas a clase única 'bug'")
for subdir in ['train', 'val']:
    for file in (ruta_dataset / 'labels' / subdir).rglob("*.txt"):
        lines = file.read_text().splitlines()
        new_lines = ['0 ' + ' '.join(line.split()[1:]) for line in lines if line.strip()]
        file.write_text('\n'.join(new_lines))

yaml_path.write_text(f"""path: {ruta_dataset.as_posix()}
train: images/train
val: images/val

nc: 1
names:
  0: bug
""")
print(f"Archivo YAML creado en: {yaml_path}")


print("Entrenando YOLOv8...")
model = YOLO(nombre_modelo)
results = model.train(data=str(yaml_path), epochs=epocas, imgsz=imgsz, device=0)

print("Analizando resultados...")
metrics_file = Path("runs/detect/train/results.csv")
if not metrics_file.exists():
    raise FileNotFoundError("No se encontró el archivo de métricas después del entrenamiento.")

df = pd.read_csv(metrics_file)

plt.figure(figsize=(14, 8))
plt.subplot(2, 2, 1)
plt.plot(df["epoch"], df["train/box_loss"], label="train/box_loss")
plt.plot(df["epoch"], df["val/box_loss"], label="val/box_loss")
plt.title("Box Loss")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.title("Precisión y Recall")
plt.legend()

"""
plt.subplot(2, 2, 3)
plt.plot(df["epoch"], df["train/obj_loss"], label="train/obj_loss")
plt.plot(df["epoch"], df["val/obj_loss"], label="val/obj_loss")
plt.title("Objectness Loss")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5")
plt.title("mAP50")
plt.legend()

"""


plt.tight_layout()
plt.show()


print(" Calculando métricas en conjunto de validación...")

val_images = list((ruta_dataset / "images/val").glob("*.jpg"))
if len(val_images) == 0:
    print(" No se encontraron imágenes en el conjunto de validación.")
else:
    preds = model.predict(val_images, save=False, conf=0.25)

    y_true = []
    y_pred = []

    for i, result in enumerate(preds):
        # Etiquetas verdaderas: clase 0 por cada objeto anotado
        label_path = ruta_dataset / "labels" / "val" / (val_images[i].stem + ".txt")
        if label_path.exists():
            true_labels = [0] * len(label_path.read_text().splitlines())
            y_true.extend(true_labels)
        else:
            continue
        pred_labels = [int(box.cls.item()) for box in result.boxes]
        y_pred.extend(pred_labels)

    if len(y_true)>0 and len(y_pred)>0:
        acc = accuracy_score(y_true, y_pred)
        bacc = balanced_accuracy_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred, labels=[0])

        print(f" Accuracy: {acc:.3f}")
        print(f" Balanced Accuracy: {bacc:.3f}")

        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["bug"])
        disp.plot(cmap="Blues")
        plt.title("Matriz de Confusión (Validación)")
        plt.show()
    else:
        print(" No se pudo calcular la matriz de confusión (sin predicciones o etiquetas válidas).")


🔄 Ajustando etiquetas a clase única 'bug'
📝 Archivo YAML creado en: /content/drive/MyDrive/BD-Ephemeroptera/data.yaml
🚀 Entrenando YOLOv8...
Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/BD-Ephemeroptera/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False

train: Scanning /content/drive/MyDrive/BD-Ephemeroptera/YOLOLO/labels/train.cache... 128 images, 14 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

train: /content/drive/MyDrive/BD-Ephemeroptera/YOLOLO/images/train/frame_oscar_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.0±1.0 ms, read: 62.4±26.6 MB/s, size: 1212.8 KB)


val: Scanning /content/drive/MyDrive/BD-Ephemeroptera/YOLOLO/labels/val.cache... 33 images, 6 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

...

                   all         33        128     0.0113      0.875      0.462      0.276

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/150      2.01G      1.377      2.077      1.195         63        640: 100%|██████████| 8/8 [00:04<00:00,  1.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.18it/s]


                   all         33        128     0.0119      0.922      0.435      0.248

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/150      2.02G      1.332      1.644      1.202         64        640: 100%|██████████| 8/8 [00:05<00:00,  1.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.07s/it]

                   all         33        128      0.012       0.93      0.559      0.317


In [ ]:
#from google.colab import files

# Entrena el modelo (ejemplo para YOLOv5)
#!python train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache

# Guarda los pesos entrenados (generalmente en runs/train/exp/weights/best.pt)
#!cp runs/train/exp/weights/best.pt /content/

# Descarga el archivo a tu computadora (opcional)
#files.download('/content/best.pt')

In [ ]:
from ultralytics import YOLO

# Entrena el modelo
model = YOLO('yolov8n.pt')  # Carga un modelo preentrenado
results = model.train(data='dataset.yaml', epochs=50, imgsz=640)

# Guarda los pesos (automáticamente se guarda en runs/detect/train/weights/best.pt)
!cp runs/detect/train/weights/best.pt /content/

# Descarga el archivo
files.download('/content/best.pt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copia los pesos a tu Drive
!cp /content/best.pt '/content/drive/MyDrive/BD-Ephemeroptera/YOLOLO/yolo_weights/'

In [ ]:
!pip install seaborn

In [1]:
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score
import numpy as np

# === CONFIGURACIÓN ===
nombre_modelo = "yolov8n.pt"  # Cambia a 'yolov8s.pt' si quieres usar un modelo un poco más grande
epocas = 20
imgsz = 640
ruta_dataset = Path(r'/home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo')#"C:/Users/Julio/Documents/tesis/Tesis-BD/efemeropteros_yolo")
yaml_path = ruta_dataset.parent / "data.yaml"

# === 1. Convertir todos los .txt a clase 0 ===
print("🔄 Ajustando etiquetas a clase única 'bug'")
for subdir in ['train', 'val']:
    for file in (ruta_dataset / 'labels' / subdir).rglob("*.txt"):
        lines = file.read_text().splitlines()
        new_lines = ['0 ' + ' '.join(line.split()[1:]) for line in lines if line.strip()]
        file.write_text('\n'.join(new_lines))

# === 2. Crear archivo YAML ===
yaml_path.write_text(f"""path: {ruta_dataset.as_posix()}
train: images/train
val: images/val

nc: 1
names:
  0: bug
""")
print(f"Archivo YAML creado en: {yaml_path}")

# === 3. Entrenar YOLO ===
print("Entrenando YOLOv8...")
model = YOLO(nombre_modelo)
results = model.train(data=str(yaml_path), epochs=epocas, imgsz=imgsz, device='cpu')

# === 4. Cargar resultados del entrenamiento ===
print("Analizando resultados...")
metrics_file = Path("runs/detect/train/results.csv")
if not metrics_file.exists():
    raise FileNotFoundError("No se encontró el archivo de métricas después del entrenamiento.")

df = pd.read_csv(metrics_file)

# === 5. Graficar pérdidas y métricas ===
plt.figure(figsize=(14, 8))
plt.subplot(2, 2, 1)
plt.plot(df["epoch"], df["train/box_loss"], label="train/box_loss")
plt.plot(df["epoch"], df["val/box_loss"], label="val/box_loss")
plt.title("Box Loss")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.title("Precisión y Recall")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(df["epoch"], df["train/obj_loss"], label="train/obj_loss")
plt.plot(df["epoch"], df["val/obj_loss"], label="val/obj_loss")
plt.title("Objectness Loss")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5")
plt.title("mAP50")
plt.legend()

plt.tight_layout()
plt.show()

# === 6. Calcular matriz de confusión ===
print("📌 Calculando métricas en conjunto de validación...")

# Ejecutar predicciones en el conjunto de validación
val_images = list((ruta_dataset / "images/val").glob("*.jpg"))
if len(val_images) == 0:
    print("⚠️ No se encontraron imágenes en el conjunto de validación.")
else:
    preds = model.predict(val_images, save=False, conf=0.25)

    y_true = []
    y_pred = []

    for i, result in enumerate(preds):
        # Etiquetas verdaderas: clase 0 por cada objeto anotado
        label_path = ruta_dataset / "labels" / "val" / (val_images[i].stem + ".txt")
        if label_path.exists():
            true_labels = [0] * len(label_path.read_text().splitlines())
            y_true.extend(true_labels)
        else:
            continue

        # Etiquetas predichas
        pred_labels = [int(box.cls.item()) for box in result.boxes]
        y_pred.extend(pred_labels)

    if y_true and y_pred:
        acc = accuracy_score(y_true, y_pred)
        bacc = balanced_accuracy_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred, labels=[0])

        print(f"✅ Accuracy: {acc:.3f}")
        print(f"✅ Balanced Accuracy: {bacc:.3f}")

        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["bug"])
        disp.plot(cmap="Blues")
        plt.title("Matriz de Confusión (Validación)")
        plt.show()
    else:
        print("⚠️ No se pudo calcular la matriz de confusión (sin predicciones o etiquetas válidas).")


🔄 Ajustando etiquetas a clase única 'bug'
Archivo YAML creado en: /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/data.yaml
Entrenando YOLOv8...
New https://pypi.org/project/ultralytics/8.3.153 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.148 🚀 Python-3.9.21 torch-2.7.0+cu126 CPU (AMD Ryzen 5 5600G with Radeon Graphics)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke

train: Scanning /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo/labels/train.cache... 128 images, 14 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

train: /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo/images/train/frame_oscar_00000.jpg: corrupt JPEG restored and saved
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 214.4±167.6 MB/s, size: 1212.8 KB)



/home/julio/anaconda3/envs/limpio39/lib/python3.9/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo/labels/val.cache... 33 images, 6 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

val: /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo/images/val/frame_oscar_00001.jpg: corrupt JPEG restored and saved
val: /home/julio/Documentos/TEST_TESIS/PracticasTesisKeras/CNN-VAE_Curso/yololo/images/val/frame_oscar_00002.jpg: corrupt JPEG restored and saved
Plotting labels to runs/detect/train2/labels.jpg... 


...

                   all         33        128      0.756      0.797      0.822      0.522



20 epochs completed in 0.162 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.148 🚀 Python-3.9.21 torch-2.7.0+cu126 CPU (AMD Ryzen 5 5600G with Radeon Graphics)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


                   all         33        128      0.756      0.797      0.821      0.522
Speed: 1.4ms preprocess, 56.7ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train2
Analizando resultados...


KeyError: 'train/obj_loss'